to start - place this in ABCD_Dir with all of your measures

In [ ]:
import pandas as pd
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from glob import glob 
import os
from os import path
import collections
import numpy as np
import matplotlib.pyplot as plt
import re
import fnmatch 
import math

edit this to point to where you have this script, and all of the ABCD Data

In [ ]:
data_dir = "/Users/work/Data/ABCD_dir/data_files"

In [ ]:
def list_slice(S, step):
    return [S[i::step] for i in range(step)]

### scrape mapping of file names to assessment name from ABCD NDAR website
MH will eventually save this and bundle it in with the script

In [ ]:
response = requests.get('https://nda.nih.gov/data_dictionary.html?source=ABCD%2BRelease%2B2.0&submission=ALL')
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
table = soup.find_all('table')[6].find_all('tr')
name_list = []

files that we don't need right now - MH will add notes on why we don't care about these measures atm

In [ ]:
ignore_files = ['abcd_cbcl01', 'image03', 'genomics_sample03', 'freesqc01', 'pasr01']
keep_files = ['abcd_mid02', 'mribrec02', 'abcd_sst02', 'abcd_mrinback02']

In [ ]:
scrape = []
name_list = []
temp_dict = collections.defaultdict(dict)

for i in range(len(table)):
    for k in table[i].find_all('td'): 
        scrape.append(k.string) 
    for k in table[i].find_all('a'):
        name_list.append(k.string)

abcd_list = list_slice(scrape, 7)
short_list = abcd_list[2]
long_list = abcd_list[1]
for i in range(len(abcd_list[0])):
    temp_dict[short_list[i]]['category'] = abcd_list[4][i]

In [ ]:
abcd_dict = collections.defaultdict(dict)
i=0
for k in short_list:
    abcd_dict[k]['name'] = long_list[i]
    if ('MRI' in long_list[i] and k not in keep_files) or (k in ignore_files) or not path.isfile(k + '.txt'): #drop all the MRI files for now
       # print(long_list[i], k)
        abcd_dict.pop(k)
    i+=1

### setup all the files we want

In [ ]:
text_files = [file + ".txt" for file in list(abcd_dict.keys())]

In [ ]:
list_of_dfs = [pd.read_csv(file, sep="\t", dtype=object, header=[0]) for file in text_files]

In [ ]:
df_dict = collections.defaultdict(dict)

for i in range(len(list_of_dfs)): 
    list_of_dfs[i]=list_of_dfs[i].set_index(list_of_dfs[i]['subjectkey']).drop(['collection_id', 'dataset_id',
                                                                                'collection_title', 'study_cohort_name',
                                                                                 'src_subject_id', 'subjectkey'], axis=1)
    key=list_of_dfs[i].columns[0][0:-3]
   # print(key, list_of_dfs[i].columns)
    score_dict = {}
    k=0
   # print(list_of_dfs[i].columns)
    for col in list(list_of_dfs[i].columns):
    #    print(i)
        #print(col, list_of_dfs[i].loc[0].tolist()[i])
        score_dict.update({col: list_of_dfs[i].iloc[0].tolist()[k]})
        k+=1
   # print(key)
    list_of_dfs[i]=list_of_dfs[i].drop(list_of_dfs[i].index[0], axis=0)
    df_dict[key]['df'] = list_of_dfs[i]
    df_dict[key]['map'] = score_dict

make them all into one dictionary! 

In [ ]:
for k in abcd_dict:
    abcd_dict[k].update(df_dict.get(k, {}))
    abcd_dict[k].update(temp_dict.get(k, {})) 

now for descriptive stats 

In [ ]:
def abcd_summary(measure, scores): 
    if len(scores) == 1:
        score = scores[0]
        data = abcd_dict[measure]['df'][score].astype(float)
        print(abcd_dict[measure]['map'][score], 'MEAN:', np.mean(data), 'STD:', np.std(data), 'MAX:', np.max(data), 'MIN:', np.min(data))
    else:
        for score in scores: 
            data = abcd_dict[measure]['df'][score].astype(float)
            print(abcd_dict[measure]['map'][score],'MEAN:', np.mean(data), 'STD:', np.std(data),'MAX:', np.max(data), 'MIN:', np.min(data))
            
def abcd_counts(measure, scores): 
    if len(scores) == 1:
        score = scores[0]
        print((abcd_dict[measure]['df'][score].value_counts()))
    else: 
        for score in scores: 
            print((abcd_dict[measure]['map'][score], abcd_dict[measure]['df'][score].value_counts()))

## neurocognition 

### handedness

In [ ]:
"""
1=right handed; 2=left handed; 3=mixed
"""
abcd_counts('abcd_ehis01', ['ehi_y_ss_scoreb'])

### scores from the neurocognition toolbox 
http://www.healthmeasures.net/explore-measurement-systems/nih-toolbox/intro-to-nih-toolbox/cognition/183-nih-toolbox-cognition

<b> Uncorrected Scores:</b>
 Uncorrected scores compare the score of the test taker to those in the nationally
representative NIH Toolbox normative sample (ages 3 to 85 years) regardless of age or any
other variable. These are presented as Standard Scores (mean=100, SD=15).
<b> Age Corrected Scores:</b> Age-corrected scores compare the score of the test-taker to others of
the same age. Age-corrected scores were derived separately for children (ages 3-17) and adults
(ages 18-85). For children, normative scores are provided separately for each year of age to
take into account expected developmental changes. These are presented as Standard Scores
(mean=100, SD=15).
<b> Fully Corrected T-Score: </b> Fully Corrected T-Scores (mean = 50, SD = 10) compare the score of
the test-taker to those in the NIH Toolbox nationally representative normative sample, while
adjusting for key demographic variables. These variables include age, gender, race/ethnicity
(white/Asian, black, Hispanic, multiracial), and educational attainment (for ages 3-17, parent’s
education is used; education is often used as a proxy for socioeconomic status). Thus a “fully
corrected” score allows for comparison within a narrower grouping.

In [ ]:
tbx_scores = [] 
for col in list(abcd_dict['abcd_tbss01']['df'].columns): 
    if fnmatch.fnmatch(col, '*nihtbx*') and not fnmatch.fnmatch(col, '*date*') and not abcd_dict['abcd_tbss01']['df'][col].isnull().all():
        tbx_scores.append(col)
        try:
            abcd_dict['abcd_tbss01']['df'][col].astype(float)
        except: 
            tbx_scores.remove(col)

abcd_summary('abcd_tbss01', tbx_scores)

### abcd cash choice task

In [ ]:
"""
1 =$75 in three days; 2 = $115 in 3 months; 3 = Don't Know
"""
abcd_dict['cct01']['df']['cash_choice_task'].replace({'1':'smaller_sooner', '2':'larger_later', '3':'do not know'}).value_counts() 

### little man task 

In [ ]:
"""
efficiency: Percentage correct divided by average reaction for correct trials
"""
#np.mean(abcd_dict['lmtp201']['df']['lmt_scr_efficiency'].astype(float))
abcd_summary('lmtp201', ['lmt_scr_efficiency'])

### pearson scores - RAVLT and WASI

In [ ]:
pearson_scores = []
for col in list(abcd_dict['abcd_ps01']['df'].columns): 
    if fnmatch.fnmatch(col, '*ravlt*') or fnmatch.fnmatch(col, '*wiscv_t*'):
        pearson_scores.append(col)

abcd_summary('abcd_ps01', pearson_scores)

### youth delay discounting

In [ ]:
delay_discount = abcd_dict['abcd_yddss01']['df'].columns[14:30]
abcd_summary('abcd_yddss01', delay_discount)

### emotional stroop

In [ ]:
emo_stroop = abcd_dict['abcd_yest01']['df'].columns[25:60]
abcd_summary('abcd_yest01', emo_stroop)

## substance abuse

### ysu02 - baseline substance use interview

### timeline followback - first use and patterns of substance useage

In [ ]:
"""
1 = yes
0 = no
"""
tlfb_knowledge = abcd_dict['abcd_ysu02']['df'].columns[8:17]
for i in tlfb_knowledge:
    abcd_dict['abcd_ysu02']['df'][i] = abcd_dict['abcd_ysu02']['df'][i].replace({'1': 'yes', '0':'no'}).copy()
    
abcd_counts('abcd_ysu02', tlfb_knowledge)

In [ ]:
"""
0 or no response = no
1 = yes
"""
tlfb_use = abcd_dict['abcd_ysu02']['df'].columns[30:83]
for i in tlfb_use: 
    if np.nansum(abcd_dict['abcd_ysu02']['df'][i].astype(float)) <= 0:
        print(i)
        tlfb_use = tlfb_use.drop(i)
    else: 
         abcd_dict['abcd_ysu02']['df'][i] = abcd_dict['abcd_ysu02']['df'][i].copy().replace({'1': 'yes', '0':'no'})
        

abcd_counts('abcd_ysu02', tlfb_use)

### ISIP alcohol measure

In [ ]:
isip_scores = abcd_dict['abcd_ysu02']['df'].columns[84:94]
abcd_counts('abcd_ysu02', isip_scores)

In [ ]:
tflb_ysu = abcd_dict['abcd_ysu02']['df'].columns[94:571]

In [ ]:
j = 0
for i in tflb_ysu: 
    try:
        if np.nansum(abcd_dict['abcd_ysu02']['df'][i].astype(float))  <= 0: 
            print(i)
            tflb_ysu = tflb_ysu.drop(i)
    except: 
            tflb_ysu = tflb_ysu.drop(i) #discard anything that can't be coerced as a float
abcd_counts('abcd_ysu02', tflb_ysu)

### xskipout - I really don't knwo what this measure is, couldn't find with google

In [ ]:
xskipout = abcd_dict['abcd_ysu02']['df'].columns[571:575]
abcd_counts('abcd_ysu02', xskipout)

### Caffeine intake

In [ ]:
caff_intake = abcd_dict['abcd_ysu02']['df'].columns[576:582]
abcd_summary('abcd_ysu02', caff_intake)
abcd_counts('abcd_ysu02', caff_intake)

### peer's thoughts on substances

In [ ]:
"""
0 = None; 1 = A Few; 2 = Some; 3 = Most; 4 = All
"""
peer_dev = abcd_dict['abcd_ysu02']['df'].columns[583:591]
abcd_counts('abcd_ysu02', peer_dev)

### alcohol useage

In [ ]:
"""
1 = Definitely yes; 2 = Probably yes; 3 = Probably not; 4 = Definitely not; 5 = Don't know; 6 = Refused to answer
"""
path_alc = abcd_dict['abcd_ysu02']['df'].columns[591:600]
abcd_counts('abcd_ysu02', path_alc)

In [ ]:
subj_resp = abcd_dict['abcd_ysu02']['df'].columns[600:612]
abcd_counts('abcd_ysu02', subj_resp)

In [ ]:
"""
none of these have responses:
xskipout_alc >= 2 && tlfb_age >=9 && tlfb_age <= 11 || xskipout_alc >= 3 && tlfb_age >=12
"""
hangover = abcd_dict['abcd_ysu02']['df'].columns[612:626]
abcd_counts('abcd_ysu02', hangover)

In [ ]:
"""
no responses here either:
xskipout_alc >= 2 && tlfb_age >=9 && tlfb_age <= 11 || xskipout_alc >= 3 && tlfb_age >=12
"""

rapi = abcd_dict['abcd_ysu02']['df'].columns[626:644]
abcd_counts('abcd_ysu02', rapi)

### nicotine useage

In [ ]:
subj_resp_nic = abcd_dict['abcd_ysu02']['df'].columns[644:653]
abcd_counts('abcd_ysu02', subj_resp_nic)

In [ ]:
path_nic = abcd_dict['abcd_ysu02']['df'].columns[653:663]
abcd_counts('abcd_ysu02', path_nic)

### marijuana useage

In [ ]:
"""
1 = Not at all; 2 = A little bit; 3 = Some; 4 = Quite a bit; 5 = A great deal
"""
sub_res_mj = abcd_dict['abcd_ysu02']['df'].columns[663:674]
abcd_counts('abcd_ysu02', sub_res_mj)

### misc. druge useage

In [ ]:
mapi = abcd_dict['abcd_ysu02']['df'].columns[674:692]
abcd_counts('abcd_ysu02', mapi)

In [ ]:
dapi = abcd_dict['abcd_ysu02']['df'].columns[692:710]
abcd_counts('abcd_ysu02', dapi)

In [ ]:
standard_units = abcd_dict['abcd_ysu02']['df'].columns[710:734]
abcd_counts('abcd_ysu02', standard_units)

### timeline followback - 1 yr followup

In [ ]:
tlfb_sum_scores = abcd_dict['abcd_tlfb01']['df'].columns[10:401]
for i in tlfb_sum_scores: 
    try: 
        if np.nansum(abcd_dict['abcd_tlfb01']['df'][i].astype(float)) <= 0:
            tlfb_sum_scores = tlfb_sum_scores.drop(i)
    except:
        tlfb_sum_scores = tlfb_sum_scores.drop(i)

abcd_counts('abcd_tlfb01', tlfb_sum_scores)

### youth participant last use - 1 yr followup

In [ ]:
ylast_use = abcd_dict['abcd_ysuip01']['df'].columns[7:500]
for i in ylast_use: 
    try: 
        if np.nansum(abcd_dict['abcd_ysuip01']['df'][i].astype(float)) <= 0:
            ylast_use = ylast_use.drop(i)
    except:
        ylast_use = ylast_use.drop(i)
abcd_counts('abcd_ysuip01', ylast_use)

### youth subsatnce use attitudes - 1 yr followup
includes path, perceived harm, peer tolerance, and expectations of substance use 

In [ ]:
attitudes = abcd_dict['abcd_ysua01']['df'].columns[5:55]
abcd_counts('abcd_ysua01', attitudes)

### youth alcohol measures - 1 yr followup

In [ ]:
alcohol_meas = abcd_dict['abcd_yam01']['df'].columns[5:56]
abcd_counts('abcd_yam01', alcohol_meas)

### youth nicotine measures - 1 yr followup

In [ ]:
nic_measures = abcd_dict['abcd_ynm01']['df'].columns[5:35]
abcd_counts('abcd_ynm01', nic_measures)

### youth mj measures - 1 yr followup

In [ ]:
mj_measures = abcd_dict['abcd_ymidm01']['df'].columns[5:58]
abcd_counts('abcd_ymidm01', mj_measures)

### youth alcohol screen - 1 yr followup

In [ ]:
"""
3 = Not applicable (not part of 10%); 1 = Negative (.000); 2 = Positive (.001 or above); 
4 = Participant unable to complete (tried 3 times); 5 = Participant refused; 6 = Not collected (other)
"""
abcd_counts('yalcs01', ['bal_test1_result_y'])
abcd_counts('yalcs01', ['bal2_test1_result_y'])

### youth toxicology - 1 yr followup

In [ ]:
"""
3 = Not applicable (not part of 10%); 1 = Yes; 2 = No; 4 = Participant unable to complete; 5 = Participant refused; 6 = Not collected (other)
"""
abcd_counts('abcd_ytt01', abcd_dict['abcd_ytt01']['df'].columns[4:30])

In [ ]:
"""
0 = NEG; 1 = POS; 2 = QNS
"""
hair_res = abcd_dict['abcd_yhr01']['df'].columns[12:33]
abcd_counts('abcd_yhr01', hair_res)

In [ ]:
nic_alert = abcd_dict['abcd_yn01']['df'].columns[5:36]
abcd_counts('abcd_yn01', nic_alert)

###  parental substance use rules 

In [ ]:
prq = abcd_dict['prq01']['df'].columns[4:19]
abcd_counts('prq01', prq)

### mid year phone interview

In [ ]:
mypi = abcd_dict['abcd_ymypisu01']['df'].columns[5:80]
abcd_counts('abcd_ymypisu01', mypi)

### parent interview: Beliefs about drug availability (alcohol, nicotine, marijuana, “other” drugs)

In [ ]:
risk_factors = abcd_dict['abcd_crpf01']['df'].columns[6:15]
abcd_counts('abcd_crpf01', risk_factors)

### summary scores substance abuse (caffeine summary scores)

In [293]:
caff = abcd_dict['abcd_suss01']['df'].columns[5:17]
abcd_summary('abcd_suss01', caff)

Sum of caff_intake_1, caff_intake_3, caff_intake_4, caff_intake_6 caff_intake_9 MEAN: 2.0717248546543465 STD: 6.526460000353603 MAX: 180.0 MIN: 0.0
Sum of caff intake. Number of Missing Answers MEAN: 0.2906947368421053 STD: 1.1636067980629627 MAX: 5.0 MIN: 0.0
Sum of caff intake:  Number of Total Questions MEAN: 5.0 STD: 0.0 MAX: 5.0 MIN: 5.0
Sum of caff_intake_1, caff_intake_3, caff_intake_4, caff_intake_6 caff_intake_9 with "0" replacing blank when tlfb_caff = "0". MEAN: 1.9548286986174976 STD: 6.358039107128077 MAX: 180.0 MIN: 0.0
Sum of caff_intake_1, caff_intake_3, caff_intake_4, caff_intake_6 caff_intake_9; Followup Annual Visits MEAN: 1.9873556172839506 STD: 7.857210084672682 MAX: 336.0 MIN: 0.0
Sum of caff_intake: Number Missing Answers; Followup Annual Visits MEAN: 0.09190062613613412 STD: 0.6716080743978677 MAX: 5.0 MIN: 0.0
Sum of caff_intakeNumber Total QuestionsFollowup Annual Visits MEAN: 5.0 STD: 0.0 MAX: 5.0 MIN: 5.0
Sum of caff_intake_1, caff_intake_3, caff_intake_4, c

## Mental Health

### ABCD Youth risk behavior survey exercise physical activity 

In [297]:
phys = abcd_dict['abcd_yrb01']['df'].columns[5:9]
abcd_summary('abcd_yrb01', phys)

During the past 7 days, on how many days were you physically active for a total of at least 60 minutes per day? (Add up all the time you spent in any kind of physical activity that increased your heart rate and made you breathe hard some of the time) MEAN: 3.493498817966903 STD: 2.3167105717156637 MAX: 7.0 MIN: 0.0
On how many of the past 7 days did you do exercises to strengthen or tone your muscles, such as push-ups, sit-ups, or weight lifting? MEAN: 2.9320888588563223 STD: 2.047897597914704 MAX: 8.0 MIN: 1.0
In an average week when you are in school, on how many days do you go to physical education (PE) class? MEAN: 2.122297297297297 STD: 1.239377619850776 MAX: 5.0 MIN: 0.0


### ABCD Youth Gender Identity - have no idea whats happening with the summary score in the mhp...

In [329]:
"""
has some flopped scoring  
first four: 
5 = Totally; 4 = Mostly; 3 = Somewhat; 2 = A little; 1 = Not at all; 0 = Decline to answer
last four: 
1 = Always; 2 = Often; 3 = Sometimes; 4 = Rarely; 5 = Never; 0 = Decline to answer
"""

gi = abcd_dict['abcd_ygi01']['df'].columns[5:13]
abcd_summary('abcd_ygi01', gi)

# gish = abcd_dict['abcd_mhp02']['df'].columns[30:36]
# abcd_summary('abcd_mhp02', gish)

How much do you feel like a boy? MEAN: 4.88073038073038 STD: 0.4789201410559225 MAX: 5.0 MIN: 0.0
How much do you feel like a girl? MEAN: 4.864024864024864 STD: 0.6685313962859294 MAX: 5.0 MIN: 0.0
How much do you feel like a girl? MEAN: 4.682452431289641 STD: 0.7294823603912435 MAX: 5.0 MIN: 0.0
How much do you feel like a boy? MEAN: 4.681606765327696 STD: 0.8004358998206357 MAX: 5.0 MIN: 0.0
How much have you had the wish to be a girl? MEAN: 4.851981351981352 STD: 0.6455287897961213 MAX: 5.0 MIN: 0.0
How much have you dressed or acted as a girl during play? MEAN: 4.7637917637917635 STD: 0.725917524170603 MAX: 5.0 MIN: 0.0
How much have you had the wish to be a boy? MEAN: 4.7775898520084565 STD: 0.7060632191317929 MAX: 5.0 MIN: 0.0
How much have you dressed or acted as a boy during play? MEAN: 4.519661733615222 STD: 0.9150689918159436 MAX: 5.0 MIN: 0.0


### ABCD Youth Life Events 
Life events including trauma

In [332]:
le = abcd_dict['abcd_mhp02']['df'].columns[7:27] #could drop every third
abcd_summary('abcd_mhp02', le)

Parent General Behavior Inventory, Sum:  Number Total Questions MEAN: 10.0 STD: 0.0 MAX: 10.0 MIN: 10.0
Total Number of Events: Validation: No Minimum MEAN: 3.1295546558704452 STD: 2.584016779482696 MAX: 19.0 MIN: 0.0
Total Number of Events: Number Missing Answers MEAN: 0.05615027267218744 STD: 1.1773111088493693 MAX: 25.0 MIN: 0.0
Total Number of Events: Number Total Questions MEAN: 25.0 STD: 0.0 MAX: 25.0 MIN: 25.0
Total Number of Good Events: Validation: No Minimum MEAN: 1.1664310240355484 STD: 1.3580301085919149 MAX: 13.0 MIN: 0.0
Total Number of Good Events: Number Total Questions MEAN: 25.0 STD: 0.0 MAX: 25.0 MIN: 25.0
Total Number of Bad Events: Validation: No Minimum MEAN: 0.9739446576449202 STD: 1.524699947358916 MAX: 12.0 MIN: 0.0
Total Number of Bad Events: Number Total Questions MEAN: 25.0 STD: 0.0 MAX: 25.0 MIN: 25.0
How Much Affected Good: Validation: No Minimum MEAN: 1.5278787878787878 STD: 2.3835146244791203 MAX: 27.0 MIN: 0.0
How Much Affected Good Sum: Number Total Qu

### KSADS